In [1]:
import numpy as np
import os
import tensorflow as tf
import zipfile
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore')

# Constants
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS = 3
EPOCHS = 20

# Extract Dataset
zip_path = "archive (1).zip"  # Change this to your actual ZIP file path
extract_path = r"C:\Users\HP\Desktop\LEAF"

if not os.path.exists(zip_path):
    print(f"Error: The file '{zip_path}' was not found.")
else:
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print("Extraction completed successfully!")
    except zipfile.BadZipFile:
        print("Error: The file is not a valid ZIP archive.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Load Dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    extract_path,
    seed=58,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

class_names = dataset.class_names
print("Class Names:", class_names)

# Split Dataset into Train, Validation, and Test
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    ds_size = len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

# Normalize pixel values to [0, 1]
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = train_ds.map(preprocess).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

# Create DenseNet121 Model
def create_densenet_model():
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
    )
    base_model.trainable = False  # Freeze pretrained layers

    model = Sequential([
        base_model,
        BatchNormalization(),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(len(class_names), activation='softmax')
    ])
    
    return model

model = create_densenet_model()
model.summary()

# Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

# Train Model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate Model
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Load Disease Information from CSV
disease_info_path = "disease_info.csv"
disease_df = pd.read_csv(disease_info_path)
disease_dict = disease_df.set_index("disease_name").to_dict(orient="index")

# Function to Predict Disease, Cause, and Prevention
def predict_leaf_disease(image_path, model):
    img = load_img(image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Shape: (1, 256, 256, 3)
    
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    disease_name = class_names[predicted_class]
    
    disease_details = disease_dict.get(disease_name, {"cause": "Unknown", "prevention": "No information available"})
    
    print("\n--- Leaf Disease Prediction ---")
    print(f"Disease: {disease_name}")
    print(f"Cause: {disease_details['cause']}")
    print(f"Prevention: {disease_details['prevention']}")
    
    return disease_name, disease_details['cause'], disease_details['prevention']

# Example Prediction
image_path = "download.jpeg"  # Replace with the actual image path
predict_leaf_disease(image_path, model)

Extraction completed successfully!
Found 3076 files belonging to 7 classes.
Class Names: ['Bacteria', 'Fungi', 'Healthy', 'Nematode', 'Pest', 'Phytopthora', 'Virus']


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)             │ (None, 8, 8, 1024)          │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 8, 8, 1024)          │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 65536)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      16,777,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │             455 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,862,471 (91.03 MB)

 Trainable params: 16,822,023 (64.17 MB)

...

77/77 ━━━━━━━━━━━━━━━━━━━━ 236s 3s/step - accuracy: 0.8849 - loss: 0.3699 - val_accuracy: 0.9340 - val_loss: 0.2107
Epoch 15/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8900 - loss: 0.3427

77/77 ━━━━━━━━━━━━━━━━━━━━ 235s 3s/step - accuracy: 0.8901 - loss: 0.3426 - val_accuracy: 0.9410 - val_loss: 0.1961
Epoch 16/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9052 - loss: 0.3310

77/77 ━━━━━━━━━━━━━━━━━━━━ 244s 3s/step - accuracy: 0.9052 - loss: 0.3309 - val_accuracy: 0.9479 - val_loss: 0.1727
Epoch 17/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 233s 3s/step - accuracy: 0.9116 - loss: 0.2959 - val_accuracy: 0.9410 - val_loss: 0.1915
Epoch 18/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 231s 3s/step - accuracy: 0.9263 - loss: 0.2506 - val_accuracy: 0.9410 - val_loss: 0.1781
Epoch 19/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9254 - loss: 0.2314

77/77 ━━━━━━━━━━━━━━━━━━━━ 234s 3s/step - accuracy: 0.9253 - loss: 0.2316 - val_accuracy: 0.9514 - val_loss: 0.1828
Epoch 20/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 234s 3s/step - accuracy: 0.9317 - loss: 0.2186 - val_accuracy: 0.9444 - val_loss: 0.1828
11/11 ━━━━━━━━━━━━━━━━━━━━ 40s 3s/step - accuracy: 0.9539 - loss: 0.1520
Test Loss: 0.16592969000339508
Test Accuracy: 0.9517045617103577
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step

--- Leaf Disease Prediction ---
Disease: Fungi
Cause: Caused by fungal spores in moist conditions
Prevention: Apply fungicides and keep leaves dry


('Fungi',
 'Caused by fungal spores in moist conditions',
 'Apply fungicides and keep leaves dry')

In [2]:
model.save('plant25.h5') 

In [3]:
from tensorflow.keras.models import load_model
loaded_model = load_model('plant25.h5')
print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [8]:
!pip install opencv-python

In [9]:
!pip install opencv-python-headless

   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.4 MB ? eta -:--:--
    --------------------------------------- 0.5/39.4 MB 229.7 kB/s eta 0:02:50
    --------------------------------------- 0.5/39.4 MB 229.7 kB/s eta 0:02:50
    --------------------------------------- 0.5/39.4 MB 229.7 kB/s eta 0:02:50
    --------------------------------------- 0.5/39.4 MB 229.7 k

In [10]:
import cv2
print(cv2.__version__)

4.11.0


In [14]:
import cv2

cap = cv2.VideoCapture(0)  
if not cap.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is working.")
    cap.release()

Camera is working.
